<a href="https://colab.research.google.com/github/tayfununal/Uniform-Autoencoder-with-Latent-Flow-Matching/blob/main/UAE-with-LFM/LFM_training/moon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/tayfununal/Uniform-Autoencoder-with-Latent-Flow-Matching.git

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch import Tensor

from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt

import os

print(os.getcwd())

%run /content/Uniform-Autoencoder-with-Latent-Flow-Matching/models/moon_model.ipynb
%run /content/Uniform-Autoencoder-with-Latent-Flow-Matching/datasets/moon_dataset.ipynb

# Call the Flow model
%run /content/Uniform-Autoencoder-with-Latent-Flow-Matching/UAE-with-LFM/models/moon_flow_model.ipynb

plt.rcParams['font.size'] = 20
plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = ['Times New Roman']

In [ ]:
# Hyper-Parameters & Settings

dataset_size = 5000
batch_size = 250


In [ ]:
# Dataset
train_dataset = MoonDataset(mode='train', n_samples=dataset_size)
val_dataset = MoonDataset(mode='val', n_samples=dataset_size)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False)

In [ ]:
# Load the UAE model

path = '/content/Uniform-Autoencoder-with-Latent-Flow-Matching/results/moon/UAE_Moon'
model = torch.load(path + '.model', weights_only=False)
model.eval()

To_Uniform(
  (encoder): Sequential(
    (0): Linear(in_features=2, out_features=64, bias=True)
    (1): SiLU()
    (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=64, out_features=64, bias=True)
    (4): SiLU()
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Linear(in_features=64, out_features=64, bias=True)
    (7): SiLU()
    (8): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Linear(in_features=64, out_features=1, bias=True)
    (10): Sigmoid()
  )
  (decoder): Sequential(
    (0): Linear(in_features=1, out_features=64, bias=True)
    (1): SiLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): SiLU()
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): SiLU()
    (6): Linear(in_features=64, out_features=2, bias=True)
    (7): Sigmoid()
  )
)

In [ ]:
# Create the "results" folder
os.makedirs("/content/Uniform-Autoencoder-with-Latent-Flow-Matching/results/moon", exist_ok=True)

flow_name = '/content/Uniform-Autoencoder-with-Latent-Flow-Matching/results/moon/UAE_Latent_FM'
flow = Flow()

optimizer = torch.optim.Adam(flow.parameters(), 0.003)
loss_fn = nn.MSELoss()

In [ ]:
flow.eval()

Flow(
  (net): Sequential(
    (0): Linear(in_features=2, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=64, bias=True)
    (7): ReLU()
    (8): Linear(in_features=64, out_features=64, bias=True)
    (9): ReLU()
    (10): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [ ]:
# --- 1. Encode the data and return (z, y) ---
def get_encoded_loader(data_loader, encoder, device='cpu'):
    encoder.eval()
    encoded = []
    labels  = []

    with torch.no_grad():
        for x, y in data_loader:  # CIFAR-10: (image, label)
            x = x.to(device)
            z = encoder(x)
            encoded.append(z.cpu())
            labels.append(y.cpu())

    all_z = torch.cat(encoded, dim=0)   # Latent vectors
    all_y = torch.cat(labels, dim=0)    # Class labels

    dataset = TensorDataset(all_z, all_y)
    return DataLoader(dataset, batch_size=data_loader.batch_size, shuffle=True)

# Keep the model on the device
model.encoder.to('cpu')

# Data loaders for encoded latents + labels
encoded_train_loader = get_encoded_loader(train_loader, model.encoder, device='cpu')
encoded_val_loader   = get_encoded_loader(val_loader, model.encoder, device='cpu')

In [ ]:
train_cost = []
val_cost = []
best_val_loss = float('inf')

for epoch in range(10000):
    epoch_losses = []

    flow.train()
    for z1, _ in encoded_train_loader:

        with torch.no_grad():
            z1 = z1
        z0 = torch.rand_like(z1)
        t = torch.rand(len(z1), 1)

        zt = (1 - t) * z0 + t * z1
        dzt = z1 - z0

        optimizer.zero_grad()
        pred = flow(t=t, x_t=zt)
        loss = loss_fn(pred, dzt)
        loss.backward()
        optimizer.step()

        epoch_losses.append(loss.item())

    mean_train_loss = sum(epoch_losses) / len(epoch_losses)
    train_cost.append(mean_train_loss)

    # Validation
    flow.eval()
    val_losses = []
    with torch.no_grad():
        for z1_val, _ in encoded_val_loader:
            z1_val = z1_val
            z0_val = torch.rand_like(z1_val)
            t_val = torch.rand(len(z1_val), 1)

            zt_val = (1 - t_val) * z0_val + t_val * z1_val
            dzt_val = z1_val - z0_val

            pred_val = flow(t=t_val, x_t=zt_val)
            val_loss = loss_fn(pred_val, dzt_val)
            val_losses.append(val_loss.item())

    mean_val_loss = sum(val_losses) / len(val_losses)
    val_cost.append(mean_val_loss)

    # Save the best model based on validation loss
    if mean_val_loss < best_val_loss:
        print("Saved!")
        best_val_loss = mean_val_loss
        torch.save(flow, flow_name + '.model')

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch {epoch+1}/10000 - Train Loss: {mean_train_loss:.4f} - Val Loss: {mean_val_loss:.4f}")

# Optionally, save the final epoch model separately
torch.save(flow, flow_name + '_final.model')

In [ ]:
# Save to CSV file
train_losses = train_cost
val_losses = val_cost

np.savetxt("/content/Uniform-Autoencoder-with-Latent-Flow-Matching/UAE-with-LFM/results/moon/losses.csv",
           np.column_stack((train_losses, val_losses)),
           delimiter=",",
           header="train_loss,val_loss",
           comments="")